In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import numpy as np
# import StringIO
import math
from keras.models import Sequential
from keras.layers import Dense, Dropout, Merge
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
np.random.seed(7)

Using Theano backend.


In [2]:
%run ../../../prepare_data.py

# import sys
# sys.path.insert(0, '../../')
# import preprare_data

In [3]:
X_train, X_other_features_train, y_train, X_validate, X_other_features_validate, y_validate = generate_full_matrix_inputs()
save_inputs('../../internal_representations/inputs/shuffeled_matrix_train_inputs_other_features_output_11.h5', X_train, y_train, other_features = X_other_features_train)
save_inputs('../../internal_representations/inputs/shuffeled_matrix_validate_inputs_other_features_output_11.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffeled_matrix_train_inputs_other_features_output_11.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffeled_matrix_validate_inputs_other_features_output_11.h5', other_features=True)

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
GENERATING X AND y...
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
GENERATION SUCCESSFUL!


In [8]:
num_examples = len(X_train) # training set size
# nn_input_dim = max_word * len(dictionary) # input layer dimensionality
# nn_output_dim = max_num_vowels * max_num_vowels # output layer dimensionality
nn_output_dim = 11
nn_hdim = 516

# Gradient descent parameters (I picked these by hand)
# epsilon = 1 # learning rate for gradient descent
# reg_lambda = 1 # regularization strength

In [9]:
word_processor = Sequential()
word_processor.add(Conv1D(43, (3), input_shape=(23, 43), padding='same', activation='relu'))
word_processor.add(Conv1D(43, (3), padding='same', activation='relu'))
word_processor.add(MaxPooling1D(pool_size=2))
word_processor.add(Flatten())
word_processor.add(Dense(516, activation='relu', kernel_constraint=maxnorm(3)))

metadata_processor = Sequential()
metadata_processor.add(Dense(256, input_dim=167, activation='relu'))

model = Sequential()
model.add(Merge([word_processor, metadata_processor], mode='concat'))  # Merge is your sensor fusion buddy
model.add(Dense(1024, input_dim=(516 + 256), activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1024, input_dim=(516 + 256), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(nn_output_dim, activation='sigmoid'))

/home/luka/miniconda3/lib/python3.5/site-packages/ipykernel_launcher.py:12: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if sys.path[0] == '':


In [10]:
# epochs = 5
# lrate = 0.1
# decay = lrate/epochs
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])


In [11]:
model.fit([X_train, X_other_features_train], y_train, validation_data=([X_validate, X_other_features_validate], y_validate), epochs=10, batch_size=10)

Train on 427470 samples, validate on 108961 samples
Epoch 1/10
 10600/427470 [..............................] - ETA: 2934s - loss: 0.0579 - acc: 0.4783

KeyboardInterrupt: 

In [9]:
model.save('v1_1.h5')

In [10]:
# model = load_model('../internal_representations/models/cnn_i2_s_c43-3relu_d20_c43-3relu_mp2_f_516relu_d20_121sigmoid_mse_adam_a65_10epoch.h5')

In [12]:
# evaluate the model
# scores = model.evaluate(X[429145:], y[429145:])
# print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

107232/107287 [============================>.] - ETA: 0s
acc: 96.52%


In [118]:
# # calculate predictions
# # test = generate_input_from_word('test', max_word, dictionary)
# # print test.shape
# # print test.T.shape
# predictions = model.predict(np.array([generate_input_from_word('biotski', max_word, dictionary)]))
# # round predictions

# print decode_position(predictions[0], max_num_vowels)

# b_pred = 0
# ind = 0
# i = 0
# for el in predictions[0]:
#     if b_pred < el:
#         b_pred = el
#         ind = i
#     i += 1
    
# print(ind)
# # rounded = [round(x[0]) for x in predictions]
# # print(rounded)

[2, 0]
2


In [23]:
def test_accuracy(predictions, y):
    dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
    num_of_pred = len(predictions)
    num_of_correct_pred = 0
    for i in range(predictions.shape[0]):
        if decode_position(predictions[i], max_num_vowels) == decode_position(y[i], max_num_vowels):
            num_of_correct_pred += 1

    return (num_of_correct_pred/float(num_of_pred)) * 100


# print(test_accuracy(predictions, y[429145:]))

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
107287
96.5186835311


In [6]:
# dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
# feature_dictionary = create_feature_dictionary(content)
def generate_input_from_word(word, max_word, dictionary):
    x = np.zeros((max_word, len(dictionary)))
    j = 0
    for c in list(word):
        index = 0
        for d in dictionary:
            if c == d:
                x[j, index] = 1
                break
            index += 1
        j += 1
    return x

# model = load_model()
# prediction = predict(model, generate_input_from_word('hidrija'))
# print decode_position(prediction[0])

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
